The task would be to recommend movies to the user based on him/her given movies.

Movies will be given by title.

In [1]:
import numpy as np
import pandas as pd

Start by inspecting our dataset

In [3]:
links_df = pd.read_csv('/content/links.csv')
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies_df = pd.read_csv('/content/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df = pd.read_csv('/content/ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags_df = pd.read_csv('/content/tags.csv')
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


Let's go on the assumption that if person $A$ likes movies $M_0, M_1, ..., M_i$,
then there goes person $B$ who likes one or more movies from $M_i$ let's call them $M_j$.

This would mean that $A$ and $B$ has a movie that they both liked, therefore other movies from both $M_j$ and $M_i$ can be liked by both $A$ and $B$ with high probability.

------------------

In [7]:
df = movies_df.merge(ratings_df, on='movieId')

In [8]:
M_j = 'Jumanji (1995)' # Title as input, now it's just one movie
recommended_movies = []

# Find the movie in the database, and sort it by rating
movie_db = df[df['title'] == M_j]\
            .sort_values(by='rating', ascending=False)

# Get the first 5 users who liked this movie
for user in movie_db.iloc[:5]['userId'].values:
    
    # Get the rated movies for this user
    rated_movies = df[df['userId'] == user]
    
    # Get the five biggest rated movie by this user
    rated_movies = rated_movies[rated_movies['title'] != M_j]\
                    .sort_values(by='rating', ascending=False)\
                    .iloc[:5]
    
    # Add these to the recommendations
    recommended_movies.extend(list(rated_movies['title'].values))
    
recommended_movies = np.unique(recommended_movies)
    
for movie in recommended_movies:
    print(movie)

Addams Family, The (1991)
Batman Forever (1995)
Crow, The (1994)
Forrest Gump (1994)
Heavy Metal (1981)
Hellraiser: Bloodline (1996)
Indiana Jones and the Temple of Doom (1984)
Interview with the Vampire: The Vampire Chronicles (1994)
Jerky Boys, The (1995)
Major Payne (1995)
Maverick (1994)
Natural Born Killers (1994)
Pan's Labyrinth (Laberinto del fauno, El) (2006)
Phenomenon (1996)
Pulp Fiction (1994)
Santa Clause, The (1994)
Secret Garden, The (1993)
Silence of the Lambs, The (1991)
Tank Girl (1995)
Toy Story (1995)
True Lies (1994)
Willow (1988)
Wyatt Earp (1994)


Now weight each movie by the similiarity on the genre feature

In [9]:
gmovie_genres = df[df['title'] == M_j].iloc[0]['genres'].split('|')
scores = {}  # {title: score ...}

for movie in recommended_movies:
    movied = df[df['title'] == movie].iloc[0]
    movie_genres = movied['genres'].split('|')
    score = 0
    
    # How many gmovie_genre can be found in movie_genres?
    for gmovie_genre in gmovie_genres:
        if gmovie_genre in movie_genres:
            score += 1
    
    scores[movie] = score
    
# Sort them on score and reverse it, because the bigger the score the better 
recommended_movies = sorted(scores, key=lambda x: scores[x])[::-1]  

The recommendations are now weighted

In [10]:
for movie in recommended_movies:
    print(movie)

Toy Story (1995)
Willow (1988)
Indiana Jones and the Temple of Doom (1984)
Addams Family, The (1991)
True Lies (1994)
Secret Garden, The (1993)
Santa Clause, The (1994)
Pan's Labyrinth (Laberinto del fauno, El) (2006)
Maverick (1994)
Heavy Metal (1981)
Crow, The (1994)
Batman Forever (1995)
Wyatt Earp (1994)
Tank Girl (1995)
Silence of the Lambs, The (1991)
Pulp Fiction (1994)
Phenomenon (1996)
Natural Born Killers (1994)
Major Payne (1995)
Jerky Boys, The (1995)
Interview with the Vampire: The Vampire Chronicles (1994)
Hellraiser: Bloodline (1996)
Forrest Gump (1994)


For implementation, see the `rmovie.py` file